# Ejercicios RDD de pares
1. Inicia las variables SparkSession y SparkContext (0)

In [1]:
import string
import sys
from pyspark import sql

session = sql.SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("01-rddej2") \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", "hdfs:///spark/logs/history") \
    .config("spark.history.fs.logDirectory", "hdfs:///spark/logs/history") \
    .getOrCreate()

sc = session.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2. Crea un rdd de pares de la forma (palabra, 1) a partir del fichero data/el_quijote.txt (1)

In [6]:
rdd = sc.textFile("hdfs:/user/jovyan/data/el_quijote.txt").flatMap(lambda linea: linea.split()).map(lambda x: (x,1))
rdd.take(10)

[('DON', 1),
 ('QUIJOTE', 1),
 ('DE', 1),
 ('LA', 1),
 ('MANCHA', 1),
 ('Miguel', 1),
 ('de', 1),
 ('Cervantes', 1),
 ('Saavedra', 1),
 ('PRIMERA', 1)]

3. A partir del RDD anterior, cuenta el número de ocurrencias de cada palabra (1)

In [13]:
word_count = rdd.reduceByKey(lambda a, b: a+b)
print(word_count.take(10))

[('DE', 17), ('LA', 13), ('Miguel', 3), ('PRIMERA', 1), ('CAPÍTULO', 1), ('1:', 1), ('condición', 23), ('y', 8042), ('del', 1113), ('hidalgo', 16)]


4. Repite los ejercicios 2 y 3 de forma que no se tengan en cuenta los signos de puntación y no se distingan mayúsculas de minúsculas (mapea todo a minúsuculas) (2)

In [15]:
rdd_fixed = rdd.map(lambda x: (x[0].lower().translate(str.maketrans('', '', string.punctuation)),x[1]))
word_count_fixed = rdd_fixed.reduceByKey(lambda a, b: a+b)
word_count_fixed.take(10)

[('no', 3077),
 ('mis', 170),
 ('alabanzas', 13),
 ('don', 1072),
 ('porque', 768),
 ('género', 29),
 ('y', 8668),
 ('ésta', 31),
 ('lo', 1806),
 ('sea', 132)]

5. A partir de los ficheros data/notas/notas_mates.txt, data/notas/notas_fisica.txt y data/notas/notas_ingles.txt, realiza las siguientes operaciones:
   - a) Crea 3 RDD de pares uno para cada asignatura (notas_mates, notas_ingles, notas_fisica) (0,5)
   - b) Crea un RDD que sea la unión de los 3 anteriores (0,5)
   - c) Muestra la nota más baja de cada alumno (1)
   - d) Muestra la nota media de cada alumno (1)
   - e) Muestra cuantos estudiantes suspenden cada asignatura (1)
   - f) Muestra en qué asignatura suspende más gente (1)
   - g) Obtén un RDD que que asocie a cada alumno con las notas en todas las asignaturas (1)

In [24]:
# a)

notas_mates = sc.textFile("hdfs:/user/jovyan/data/notas/notas_mates.txt").map(lambda linea: (linea.split(',')[0], float(linea.split(',')[1])))
notas_ingles = sc.textFile("hdfs:/user/jovyan/data/notas/notas_ingles.txt").map(lambda linea: (linea.split(',')[0], float(linea.split(',')[1])))
notas_fisica = sc.textFile("hdfs:/user/jovyan/data/notas/notas_fisica.txt").map(lambda linea: (linea.split(',')[0], float(linea.split(',')[1])))
notas_mates.collect()



[('Angel', 6.0),
 ('Maria', 2.0),
 ('Ramon', 4.5),
 ('Jorge', 10.0),
 ('Susana', 9.0),
 ('Anabel', 8.0),
 ('Pedro', 5.0),
 ('Rocio', 6.0),
 ('Carlos', 4.0),
 ('Triana', 3.0),
 ('Andres', 4.0),
 ('Fernando', 5.0),
 ('Leonardo', 1.0),
 ('Oscar', 7.0),
 ('Isabel', 8.0),
 ('Jose Juan', 5.0),
 ('Nicolas', 2.0),
 ('Alejandro', 5.0),
 ('Rosa', 6.0)]

In [25]:
# b)
rdd_union = notas_mates.union(notas_fisica).union(notas_ingles)

rdd_union.take(10)

[('Angel', 6.0),
 ('Maria', 2.0),
 ('Ramon', 4.5),
 ('Jorge', 10.0),
 ('Susana', 9.0),
 ('Anabel', 8.0),
 ('Pedro', 5.0),
 ('Rocio', 6.0),
 ('Carlos', 4.0),
 ('Triana', 3.0)]

In [29]:
# c)
print(f"Nota más baja de cada alumno: {rdd_union.reduceByKey(lambda x,y : min(x,y)).collect()}")

[Stage 19:=============================>                            (3 + 2) / 6]

Nota más baja de cada alumno: [('Angel', 4.0), ('Carlos', 4.0), ('Anabel', 2.0), ('Jose Juan', 3.0), ('Andres', 4.0), ('Jorge', 5.0), ('Susana', 2.0), ('Fernando', 5.0), ('Oscar', 3.0), ('Isabel', 7.0), ('Alejandro', 3.0), ('Rocio', 4.0), ('Leonardo', 1.0), ('Nicolas', 2.0), ('Ramon', 4.5), ('Pedro', 2.0), ('Maria', 2.0), ('Triana', 3.0), ('Rosa', 6.0)]


In [36]:
# d)
print(f"Nota media de cada alumno: {rdd_union.groupByKey().mapValues(lambda notas: sum(notas)/len(notas)).collect()}")


Nota media de cada alumno: [('Angel', 6.333333333333333), ('Carlos', 5.333333333333333), ('Anabel', 5.666666666666667), ('Jose Juan', 3.6666666666666665), ('Jorge', 6.666666666666667), ('Susana', 6.666666666666667), ('Andres', 4.666666666666667), ('Rocio', 5.5), ('Fernando', 7.0), ('Oscar', 5.0), ('Isabel', 7.666666666666667), ('Alejandro', 5.0), ('Leonardo', 3.6666666666666665), ('Nicolas', 4.666666666666667), ('Ramon', 6.5), ('Pedro', 3.5), ('Maria', 3.6666666666666665), ('Triana', 3.3333333333333335), ('Rosa', 7.666666666666667)]


In [ ]:
# e)
suspenden_mates = notas_mates_rdd.filter(lambda x: x[1] < 5).count()
suspenden_fisica = notas_fisica_rdd.filter(lambda x: x[1] < 5).count()
suspenden_ingles = notas_ingles_rdd.filter(lambda x: x[1] < 5).count()
print(f"Estudiantes que suspenden Matemáticas: {suspenden_mates}")
print(f"Estudiantes que suspenden Física: {suspenden_fisica}")
print(f"Estudiantes que suspenden Inglés: {suspenden_ingles}")

In [ ]:
# f)

if suspenden_mates > suspenden_fisica and suspenden_mates > suspenden_ingles:
    print(f"La asignatura con más suspendidos es Matemáticas")
elif suspenden_fisica > suspenden_mates and suspenden_fisica > suspenden_ingles:
    print(f"La asignatura con más suspendidos es Física")
else:
    print(f"La asignatura con más suspendidos es Inglés.")

In [38]:
# g)
rdd_total = notas_mates.join(notas_fisica).join(notas_ingles).mapValues(lambda t: (t[0][0], t[0][1], t[1]))
rdd_total.collect()

[('Carlos', (4.0, 4.0, 8.0)),
 ('Angel', (6.0, 9.0, 4.0)),
 ('Anabel', (8.0, 2.0, 7.0)),
 ('Jose Juan', (5.0, 3.0, 3.0)),
 ('Jorge', (10.0, 5.0, 5.0)),
 ('Susana', (9.0, 9.0, 2.0)),
 ('Andres', (4.0, 4.0, 6.0)),
 ('Oscar', (7.0, 5.0, 3.0)),
 ('Rocio', (6.0, 5.0, 4.0)),
 ('Rocio', (6.0, 7.0, 4.0)),
 ('Fernando', (5.0, 9.0, 7.0)),
 ('Isabel', (8.0, 8.0, 7.0)),
 ('Alejandro', (5.0, 3.0, 7.0)),
 ('Leonardo', (1.0, 6.0, 4.0)),
 ('Nicolas', (2.0, 7.0, 5.0)),
 ('Ramon', (4.5, 7.0, 8.0)),
 ('Maria', (2.0, 3.0, 6.0)),
 ('Triana', (3.0, 3.0, 4.0)),
 ('Rosa', (6.0, 8.0, 9.0))]